# Checkpoint 01 – Análise de Dados de Consumidores de Energia

Este notebook segue os passos de exploração e análise propostos na atividade de checkpoint. Ele utiliza o dataset **Individual Household Electric Power Consumption** do UCI Machine Learning Repository (2006–2010) para responder às questões e gerar insights sobre o consumo elétrico domiciliar.

Lembre-se de colocar o arquivo `household_power_consumption.txt` na raiz do repositório antes de executar as células abaixo.


## 1. Carregar o dataset e exibir as 10 primeiras linhas

In [ ]:
import pandas as pd

# Ajuste o caminho se necessário
file_path = 'household_power_consumption.txt'

# Carregamos o arquivo; valores ausentes estão representados por '?'
df = pd.read_csv(
    file_path,
    sep=';',
    na_values='?',
    low_memory=False
)

# Exibimos as 10 primeiras linhas
df.head(10)


## 2. Diferença entre `Global_active_power` e `Global_reactive_power`

* **Global_active_power** representa a potência elétrica ativa média (kW) consumida pela residência em cada minuto. Essa é a energia efetivamente transformada em trabalho útil (como iluminação, aquecimento, eletrodomésticos etc.).
* **Global_reactive_power** representa a potência elétrica reativa média (kVAR). Ela está associada aos campos magnéticos gerados por componentes indutivos ou capacitivos (motores, transformadores), não realizando trabalho útil — apenas circula entre a fonte e as cargas para manter os campos magnéticos.


## 3. Verificar valores ausentes no dataset

In [ ]:
# Contar valores ausentes por coluna
missing_counts = df.isna().sum()
missing_counts


## 4. Converter `Date` para datetime e criar coluna com o dia da semana

In [ ]:
# Criamos uma coluna datetime combinando Date e Time
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S')
# Ajustamos o índice
# df = df.set_index('Datetime')

# Criamos a coluna com nome do dia da semana
df['weekday'] = df['Datetime'].dt.day_name()

df[['Datetime', 'weekday']].head()


## 5. Filtrar registros de 2007 e calcular a média diária de `Global_active_power`

In [ ]:
# Converter Global_active_power para numérico
df['Global_active_power'] = pd.to_numeric(df['Global_active_power'], errors='coerce')

# Filtrar ano de 2007
df_2007 = df[df['Datetime'].dt.year == 2007]

# Calcular média diária
daily_mean_2007 = df_2007.set_index('Datetime')['Global_active_power'].resample('D').mean()

daily_mean_2007


## 6. Variação de `Global_active_power` em um dia escolhido

In [ ]:
import matplotlib.pyplot as plt

# Selecionamos um dia com dados completos (ex.: 15 de março de 2008)
day = df[(df['Datetime'].dt.date == pd.to_datetime('2008-03-15').date())].copy()

plt.figure()
day['Global_active_power'].plot()
plt.title('Global Active Power em 15/03/2008')
plt.xlabel('Horário')
plt.ylabel('Potência Ativa Global (kW)')
plt.show()


## 7. Histograma da variável `Voltage`

In [ ]:
# Converter Voltage para numérico
df['Voltage'] = pd.to_numeric(df['Voltage'], errors='coerce')

plt.figure()
df['Voltage'].hist(bins=100)
plt.title('Histograma de Voltage')
plt.xlabel('Voltage (V)')
plt.ylabel('Frequência')
plt.show()

# Estatísticas descritivas
df['Voltage'].describe()


## 8. Consumo médio mensal ao longo de todo o período

In [ ]:
# Calcular média mensal de Global_active_power
monthly_mean = df.set_index('Datetime')['Global_active_power'].resample('M').mean()

monthly_mean


## 9. Dia com maior consumo de energia ativa global

In [ ]:
# Soma diária de Global_active_power
daily_sum = df.set_index('Datetime')['Global_active_power'].resample('D').sum()

max_day = daily_sum.idxmax()
max_value = daily_sum.max()

max_day, max_value


## 10. Comparação do consumo médio entre dias de semana e finais de semana

In [ ]:
# Classificação de fim de semana
is_weekend = df['Datetime'].dt.weekday >= 5

weekday_mean = df.loc[~is_weekend, 'Global_active_power'].mean()
weekend_mean = df.loc[is_weekend, 'Global_active_power'].mean()

weekday_mean, weekend_mean


## 11. Correlação entre variáveis selecionadas

In [ ]:
# Selecionamos as colunas de interesse
df['Global_reactive_power'] = pd.to_numeric(df['Global_reactive_power'], errors='coerce')
df['Global_intensity'] = pd.to_numeric(df['Global_intensity'], errors='coerce')

corr_matrix = df[['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity']].corr()
corr_matrix


## 12. Nova variável `Total_Sub_metering`

In [ ]:
# Converter sub_metering para numérico
df['Sub_metering_1'] = pd.to_numeric(df['Sub_metering_1'], errors='coerce')
df['Sub_metering_2'] = pd.to_numeric(df['Sub_metering_2'], errors='coerce')
df['Sub_metering_3'] = pd.to_numeric(df['Sub_metering_3'], errors='coerce')

# Criar a soma

df['Total_Sub_metering'] = df[['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']].sum(axis=1)

df['Total_Sub_metering'].head()


## 13. Meses em que `Total_Sub_metering` ultrapassa a média de `Global_active_power`

In [ ]:
monthly_tot_sub = df.set_index('Datetime')['Total_Sub_metering'].resample('M').mean()
monthly_ga = df.set_index('Datetime')['Global_active_power'].resample('M').mean()
exceed_months = monthly_tot_sub[monthly_tot_sub > monthly_ga]
exceed_months


## 14. Série temporal do `Voltage` para o ano de 2008

In [ ]:
voltage_2008 = df[df['Datetime'].dt.year == 2008].set_index('Datetime')['Voltage']

plt.figure()
voltage_2008.plot()
plt.title('Voltage em 2008')
plt.xlabel('Data')
plt.ylabel('Voltage (V)')
plt.show()


## 15. Comparação do consumo entre meses de verão e inverno (hemisfério norte)

In [ ]:
summer_mean = df[df['Datetime'].dt.month.isin([6,7,8])]['Global_active_power'].mean()
winter_mean = df[df['Datetime'].dt.month.isin([12,1,2])]['Global_active_power'].mean()

summer_mean, winter_mean


## 16. Amostragem aleatória de 1% e comparação da distribuição de `Global_active_power`

In [ ]:
sample_df = df.sample(frac=0.01, random_state=42)

# Estatísticas descritivas
df['Global_active_power'].describe(), sample_df['Global_active_power'].describe()

# (Opcional) Histograma comparativo
import numpy as np

plt.figure()

# histograma da base completa
plt.hist(df['Global_active_power'].dropna(), bins=100, density=True, alpha=0.5, label='Completa')
# histograma da amostra
plt.hist(sample_df['Global_active_power'].dropna(), bins=100, density=True, alpha=0.5, label='Amostra 1%')
plt.legend()
plt.title('Comparação de distribuições: completa vs amostra 1%')
plt.xlabel('Global Active Power (kW)')
plt.ylabel('Densidade')
plt.show()


## 17. Normalização (Min-Max Scaling) das variáveis numéricas principais

In [ ]:
from sklearn.preprocessing import MinMaxScaler

numeric_cols = ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity',
               'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Total_Sub_metering']

scaler = MinMaxScaler()
# Preencher NaNs com 0 antes de ajustar
scaled_data = scaler.fit_transform(df[numeric_cols].fillna(0))
scaled_df = pd.DataFrame(scaled_data, columns=numeric_cols)

scaled_df.describe().loc[['min','max','mean','std']]


## 18. K-means para segmentar dias em 3 grupos de consumo

In [ ]:
from sklearn.cluster import KMeans

# Agregar dados por dia
daily_features = pd.DataFrame({
    'mean_ga': df.set_index('Datetime')['Global_active_power'].resample('D').mean(),
    'mean_gr': df.set_index('Datetime')['Global_reactive_power'].resample('D').mean(),
    'mean_intensity': df.set_index('Datetime')['Global_intensity'].resample('D').mean(),
    'sum_sub': df.set_index('Datetime')['Total_Sub_metering'].resample('D').sum(),
}).dropna()

kmeans = KMeans(n_clusters=3, random_state=42)
labels = kmeans.fit_predict(daily_features)
daily_features['cluster'] = labels

cluster_counts = daily_features['cluster'].value_counts()
centers = pd.DataFrame(kmeans.cluster_centers_, columns=daily_features.columns[:-1])

cluster_counts, centers


## 19. Decomposição de série temporal (tendência, sazonalidade e resíduo)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Selecionar seis meses de dados (jan–jun 2007) e agregar por dia
half_year = df[(df['Datetime'] >= '2007-01-01') & (df['Datetime'] <= '2007-06-30')]
series = half_year.set_index('Datetime')['Global_active_power'].resample('D').mean().dropna()

result = seasonal_decompose(series, model='additive', period=7)
result.plot()
plt.suptitle('Decomposição sazonal da Global Active Power (jan–jun 2007)')
plt.show()


## 20. Regressão linear simples para prever `Global_active_power` a partir de `Global_intensity`

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Selecionar linhas válidas
valid = df[['Global_active_power', 'Global_intensity']].dropna()
X = valid[['Global_intensity']]
y = valid['Global_active_power']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)

preds = lr.predict(X_test)

rmse = mean_squared_error(y_test, preds, squared=False)
mae = mean_absolute_error(y_test, preds)
r2 = lr.score(X_test, y_test)

coefficient, intercept = lr.coef_[0], lr.intercept_

coefficient, intercept, rmse, mae, r2
